In [1]:
#%matplotlib inline
import sys
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/weather_mast/')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/MEPS/')
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.style.use('ggplot')
import datetime
import math

import createFolder as cF
import calc_date as cd
import plot_sfc_spaghetti_ret as spagh
import save_fig as SF
import get_Haukeli_obs_data as obsDat
import calc_48h_acc as acc
import fill_values as fv
import plot_vertical as pvert

import os

import pandas as pd

In [2]:
year = 2016
month = 12
tid = '00'
#tid = '18'
station = 'Haukeliseter'
savefig = 1  # 1 = yes, 0 = no

### if plot with double fence data
dofe = 1     # 1 = yes, 0 = no

In [3]:
#### days 
#t = ['17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27']
t = ['20', '21', '22', '23', '24', '25', '26', '27']
#t = ['22']

In [4]:
var_name = 'precipitation_amount_acc'
unit = '[mm]'
figdir = '../../Figures/Retrieval_MEPS/boxplot' 
# Folder where the figures are being saved    
cF.createFolder('%s/' %(figdir))
form = 'png'

In [5]:
level = 'sfc'
### MEPS
ncdir_sfc = '../../Data/MEPS/%s/%s_%s' %(station,level,tid)
level = '10m'
ncdir_10m = '../../Data/MEPS/%s/%s_%s' %(station,level,tid)


################################################################
### Haukeli obs.
txtdir = '../../Data/eklima/'
txt_filename = 'Observations-hour_minutes'

################################################################
### Retrieval 
nc_dir_retrieval = '../../Data/Retrieved_SWC'

################################################################



In [6]:
# read in the Haukeliseter observation file, sorted by daily values
Haukeli = pd.read_csv('%s/%s.txt' %(txtdir, txt_filename),\
                    sep = ';',header=19, skipfooter = 11, engine= 'python')

DateHour = Haukeli['Date-Hour(UTC)']

RR_1 = Haukeli['RR_1'].astype(float)           # Precipitation (1 hour); [mm]
DD   = Haukeli['DD'].astype(float)             # Wind direction (FF); [degrees]
FF   = Haukeli['FF'].astype(float)             # Wind speed (10 meters above ground);m/s

if tid == '18':
    precip, dt_precip, hour_precip, day_precip, month_precip, year_precip = obsDat.valid_values_18UTC(RR_1, DateHour)
    dt, p, hour, dy, mm, yr = obsDat.arange_daily_18UTC(RR_1, DateHour)
if tid == '00':
    precip, dt_precip, hour_precip, day_precip, month_precip, year_precip = obsDat.valid_values(RR_1, DateHour)
    dt, p, hour, dy, mm, yr = obsDat.arange_daily(RR_1, DateHour)

   

In [7]:
idxFF = np.where(FF[:] == -9999)
idxDD = np.where(DD[:] == -9999)

FF[idxFF[0][:]] = np.nan
DD[idxDD[0][:]] = np.nan

### calculate the U, V wind component for barb plot
# http://colaweb.gmu.edu/dev/clim301/lectures/wind/wind-uv.html

# first calculate the mathematical wind direction in deg
md_deg = 270 - DD
for k in range(0,md_deg.shape[0]):
    if md_deg[k] <0 :
        md_deg[k] = md_deg[k] +360
md_rad = math.pi/180. * md_deg
uwind = FF*np.cos(md_rad)
vwind = FF*np.sin(md_rad)

In [8]:
U_wind, V_wind, dt_wind, hour_wind, day_wind, month_wind, year_wind = obsDat.valid_values_wind(uwind, vwind, DateHour)

In [9]:
# read in the Haukeliseter observation file, for wind gust
Haukeli_gust = pd.read_csv('%s/%s_gusts.txt' %(txtdir, txt_filename),\
                    sep = ';',header=14, skipfooter = 2,engine= 'python')

In [10]:
# FG_1 Kraftigste vindkast (siste time)
# FX_1 Kraftigste middelvind (siste time)
FX_1 = Haukeli_gust['FX_1'].astype(float)
DX_1 = Haukeli_gust['DX_1'].astype(float)

In [11]:
idxFX_1 = np.where(FX_1[:] == -9999)
idxDX_1 = np.where(DX_1[:] == -9999)

FX_1[idxFX_1[0][:]] = np.nan
DX_1[idxDX_1[0][:]] = np.nan

# first calculate the mathematical wind direction in deg
md_deg_X = 270 - DX_1
for k in range(0,md_deg_X.shape[0]):
    if md_deg_X[k] <0 :
        md_deg_X[k] = md_deg_X[k] +360
md_rad_X = math.pi/180. * md_deg_X
uwind_X = FX_1*np.cos(md_rad_X)
vwind_X = FX_1*np.sin(md_rad_X)

In [12]:
U_wind_X, V_wind_X, dt_wind_X, hour_wind_X, day_wind_X, month_wind_X, year_wind_X = obsDat.valid_values_wind(uwind_X, vwind_X, DateHour)

In [13]:
fn_sfc        = dict()
fn_10m        = dict()
time_sfc      = dict()
ini_day       = []
calday        = []
calmon        = []
precipitation_amount_acc = dict()
x_wind_10m    = dict()
y_wind_10m    = dict()
lead_time_sfc = dict()
title         = dict()
snowfall_amount = dict()
rainfall_amount = dict()
graupelfall_amount = dict()
rain_snow_graup = dict()

air_temp = dict()
pressure = dict()
rain_snow_graup = dict()
density = dict()
SWE = dict()

Rd = 287.

In [14]:
#t = [25]
for day in t:


    var_name_sfc = '%s' %(var_name)

    ## RETRIEVAL ######
    fn_ret1 = netCDF4.Dataset('%s/out_swc_b6_%s%s%s.nc' %(nc_dir_retrieval, year, month, day))
    fn_ret2 = netCDF4.Dataset('%s/out_swc_b6_%s%s%s.nc' %(nc_dir_retrieval, year, month, int(day)+1))
    equiv_accum1 = fn_ret1.variables['retrieved_snowfall_amount'][:,:] 
    equiv_accum2 = fn_ret2.variables['retrieved_snowfall_amount'][:,:] 


### calculate accumulation from retrieval
#srate = equiv_accum1[6,:]
    srate1 = (equiv_accum1[2,:])
    srate2 = (equiv_accum2[2,:])
    

#srate[np.where(~np.isfinite(srate))] = 0.0
#srate = srate/srate.shape[0]
    srate1[np.where(~np.isfinite(srate1))] = 0.0
    srate2[np.where(~np.isfinite(srate2))] = 0.0
    srate1 = srate1/srate1.shape[0]
    srate2 = srate2/srate2.shape[0]
    srate = np.concatenate((srate1, srate2),axis=0)
#pd.DataFrame(srate)

    acc_ret = []
    acc_ret.append(0.0)
    for i in range(0,srate.shape[0]-1):
        dix = np.nansum([acc_ret[i],srate[i]],axis = 0)
        acc_ret.append(dix)

    for ens_memb in range(0,10):
# read in the netcdf surface file
        fn_sfc[ens_memb] = netCDF4.Dataset('%s/%s%s%s_%s_%s.nc' %(ncdir_sfc,year,month,day,tid,ens_memb))
        time_sfc[ens_memb] = fn_sfc[ens_memb].variables['time']
        time_sfc[ens_memb] = fv.fill_nan(time_sfc[ens_memb][:])
# initialisation time for plotting
        ini_day = (datetime.datetime.utcfromtimestamp(time_sfc[0][0]).day)   # day of initialisation
        hh = (datetime.datetime.utcfromtimestamp(time_sfc[0][0]).hour)       # first hour of initialisation
        calday, calmon = cd.get_dayname(year, month, ini_day)

# Read in the pressure, and the air temperature to calculate ice amount from [kg/kg] to [kg/m^3]
        air_temp[ens_memb] = fn_sfc[ens_memb].variables['air_temperature_0m']
        pressure[ens_memb] = fn_sfc[ens_memb].variables['surface_air_pressure']
        
# Read in the variable name which should be plotted  
        precipitation_amount_acc[ens_memb] = fn_sfc[ens_memb].variables[var_name]
 
    
# read in the netcdf 10m wind file
        fn_10m[ens_memb] = netCDF4.Dataset('%s/%s%s%s_%s_%s.nc' %(ncdir_10m,year,month,day,tid,ens_memb))
# read in the wind variable
        x_wind_10m[ens_memb] = fn_10m[ens_memb].variables['x_wind_10m'][:]
        y_wind_10m[ens_memb] = fn_10m[ens_memb].variables['y_wind_10m'][:]
    



# lead time
        lead_time_sfc[ens_memb] = np.arange(0,time_sfc[ens_memb].shape[0])
# get only valuable values not nan
        #substitute missing values with nan 
        pressure[ens_memb]       = fv.fill_nan(pressure[ens_memb][:])
        air_temp[ens_memb]       = fv.fill_nan(air_temp[ens_memb][:])
        
        precipitation_amount_acc[ens_memb] = fv.fill_nan(precipitation_amount_acc[ens_memb][:,:])

    

    val = np.zeros(shape=(10,precipitation_amount_acc[0].shape[0]))
    ice_EM = []
    ice_mean = []
    ice_median = []
    ice_75 = []
    ice_25 = []
    ice_max = []
    ice_min = []
    variable = []
# calculate the ensemble mean of ice amount (all 10 ensemble member averaged)
#    for k in range(0,ice_amount_ml[0].shape[0]):
#for k in range(0,25):
#    for k in range(0,1):
    for ens_memb in range(0,10):
            val[ens_memb] = ((precipitation_amount_acc[ens_memb])[:,0])
    variable.append(val)
    ### 75% quartile
    ice_mean.append(np.nanmean(val,axis=0))  
#############################################
# Create a figure instance
    
    fig_name = '%s%s%s_%s.png' %(year,month,ini_day,hh)
    fig = plt.figure(1, figsize=(20,7))

# Create an axes instance
    ax = fig.add_subplot(111)
    
# Vertical line to show end of day
    ax.axvline(0+1,color = spagh.vert_col, linewidth = 3)
    ax.axvline(24+1,color = spagh.vert_col, linewidth = 3)
    ax.axvline(48+1,color = spagh.vert_col, linewidth = 3)
    
    bp = ax.boxplot(np.asarray(variable)[0,:,:49],0,'',patch_artist=True)
#    ax.plot(np.arange(0,50,3),np.asarray(ice_mean)[0,0:3:49])
    ax.plot(np.arange(1,np.asarray(ice_mean).size+1,3),np.asarray(ice_mean)[0,::3])

## change outline color, fill color and linewidth of the boxes
    for box in bp['boxes']:
    # change outline color
        box.set( color='#7570b3', linewidth=2)
    # change fill color
        box.set( facecolor = '#1b9e77' )

## change color and linewidth of the whiskers
    for whisker in bp['whiskers']:
        whisker.set(color='#7570b3', linewidth=2)

## change color and linewidth of the caps
    for cap in bp['caps']:
        cap.set(color='#7570b3', linewidth=2)

## change color and linewidth of the medians
    for median in bp['medians']:
        median.set(color='#b2df8a', linewidth=2)

## change the style of fliers and their fill
    for flier in bp['fliers']:
        flier.set(marker='o', color='#e7298a', alpha=0.5)
    

    vari_name = var_name_sfc.split("_")
# yaxis
    ax.set_ylabel('%s %s %s' %(vari_name[0], vari_name[1], unit), fontsize = spagh.label_fs)
    ax.set_ylim(-0.5,80)
    T = np.arange(0,90,10)
    ax.set_yticks(T)
    ax.set_yticklabels(T,fontsize = spagh.fontsize)

    Xmax = 49
# xaxis
    a = lead_time_sfc[0][0:49]
    
    ax.set_xticks(np.arange(0,Xmax,6))
#plt.setp(ax.get_xticklabels(), visible=False) 
    ax.set_xticks(lead_time_sfc[0][:(Xmax)])
    ax.set_xlim([0.5,Xmax+0.5])
    ax.set_xticks(np.arange(1,Xmax+1,6))

    h_p18 = hour
    m_p18 = mm 
    d_p18 = dy 
    y_p18 = yr
    ini_day = int(ini_day)
    dates = pvert.dates_plt_00(h_p18, m_p18, d_p18, y_p18, ini_day)
    ax.tick_params(axis='both', which= 'major', labelsize=24)

    ax.set_xticks(np.arange(1,Xmax+1,6))
    ax.set_xticklabels(dates, rotation = 25, fontsize = spagh.tick_fs)
    ax.set_xlabel('time', fontsize = spagh.label_fs)
# title
    if hh < 10:
        hh = '0%s' %(hh)
    else:
        hh = '%s' %(hh)
        
    title = 'initalised: %s, %s %s %s %s UTC' %(calday,ini_day,calmon,year,hh)
    ax.set_title(title, fontsize=spagh.fontsize, color =spagh.blue )
# tight layout
    plt.tight_layout() 

## Remove top axes and right axes ticks
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()

    if savefig == 1:
        SF.save_figure_landscape(figdir, fig_name, form)
        print('plot saved: %s/%s' %(figdir,fig_name))
    elif savefig == 0:
        plt.show()
    plt.close()
    

        
#    for ens_memb in range(0,10):
 #       fn_sfc[ens_memb].close()

/Volumes/SANDISK128/Applications/anaconda/envs/py36wind/lib/python3.6/site-packages/ipykernel_launcher.py:90: RuntimeWarning: Mean of empty slice
/Volumes/SANDISK128/Applications/anaconda/envs/py36wind/lib/python3.6/site-packages/numpy/lib/function_base.py:4116: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)
/Volumes/SANDISK128/Applications/anaconda/envs/py36wind/lib/python3.6/site-packages/matplotlib/cbook.py:2066: RuntimeWarning: invalid value encountered in less_equal
  wiskhi = np.compress(x <= hival, x)
/Volumes/SANDISK128/Applications/anaconda/envs/py36wind/lib/python3.6/site-packages/matplotlib/cbook.py:2073: RuntimeWarning: invalid value encountered in greater_equal
  wisklo = np.compress(x >= loval, x)
/Volumes/SANDISK128/Applications/anaconda/envs/py36wind/lib/python3.6/site-packages/matplotlib/cbook.py:2081: RuntimeWarning: invalid value encountered in less
  np.compress(x < stats['whislo'], x),
/Volumes/SANDISK128/Applications/anacond

plot saved: ../../Figures/Retrieval_MEPS/boxplot/20161220_0.png
plot saved: ../../Figures/Retrieval_MEPS/boxplot/20161221_0.png
plot saved: ../../Figures/Retrieval_MEPS/boxplot/20161222_0.png
plot saved: ../../Figures/Retrieval_MEPS/boxplot/20161223_0.png
plot saved: ../../Figures/Retrieval_MEPS/boxplot/20161224_0.png
plot saved: ../../Figures/Retrieval_MEPS/boxplot/20161225_0.png
plot saved: ../../Figures/Retrieval_MEPS/boxplot/20161226_0.png
plot saved: ../../Figures/Retrieval_MEPS/boxplot/20161227_0.png
